### ### final3_version3 - 처리 사항
- matchType 통합

- 변수제거
> groupId,matchId,killplace,(killpoint, rankpoint, winpoint => Weighted), numgroups, maxplace

- 이상치 제거
> matchType은 이상치 없음<br>
> 'assists_mean', 'boosts_mean','headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean','longestKill_mean', 'rideDistance_mean','swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','DBNOs_mean','revives_mean' => quantile 상위 0.005퍼센트 이상치 제거<br>
> 평균값이기 때문에 total distance==0 제거(핵쟁이일 가능성)

- 데이터 변환
> distance 변수 sum
> matchId, groupId 별로 평균값으로 컬럼 대체<br>
> match 평균 변수 추가<br>
> matchsize 변수 추가<br>
> 각_mean 변수 / 각_match_mean 변수 => 컬럼개수 줄이고, 해당 판에서 그 사람의 위치를 비율로 만들어줄 수 있음

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm_notebook
import sklearn.preprocessing
import gc

In [2]:
# dataframe display 옵션
pd.set_option('display.max_columns' , 100)
pd.set_option('display.width', None)

In [3]:
df_train = pd.read_csv('./data/test_V2.csv', dtype={
                    'assists': 'int16',
                    'boosts': 'int16',
                    'DBNOs': 'int16',
                    'headshotKills': 'int16',
                    'heals': 'int16',
                    'kills': 'int16',
                    'killStreaks': 'int16',
                    'longestKill': 'float32',
                    'revives': 'int16',
                    'rideDistance': 'float32',
                    'swimDistance': 'float32',
                    'vehicleDestroys': 'int16',
                    'walkDistance': 'float32',
                    'weaponsAquired': 'int16'
                })

In [4]:
# 필요 없는 칼럼 제거
del_list = ['maxPlace','killPlace', 'killPoints',
            'numGroups', 'rankPoints','winPoints','matchDuration']
for col in del_list :
    del df_train[col]

In [5]:
features = list(df_train.columns)    
    
df_train_mean = df_train.groupby(['matchId','groupId']).mean()
df_train_size = df_train.groupby(['matchId']).size().reset_index(name='match_size')
df_train_match_mean = df_train.groupby(['matchId']).mean()
df_train_max = df_train.groupby(['matchId','groupId'])[features].agg('max')
df_train_min = df_train.groupby(['matchId','groupId'])[features].agg('min')

df_train = pd.merge(df_train, df_train_max, suffixes=["", "_max"], how='left', on=['matchId', 'groupId'])
del df_train_max
df_train = pd.merge(df_train, df_train_min, suffixes=["", "_min"], how='left', on=['matchId', 'groupId'])
del df_train_min
df_train = pd.merge(df_train, df_train_mean, suffixes=["","_mean"], how = 'left', on=['matchId','groupId'])
del df_train_mean
df_train = pd.merge(df_train,df_train_match_mean, suffixes=["","_match_mean"], how="left", on=['matchId'])
del df_train_match_mean
df_train = pd.merge(df_train, df_train_size, how='left', on=['matchId'])
del df_train_size

In [6]:
df_train.head(10)

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,kills,killStreaks,longestKill,matchType,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,DBNOs_max,Id_max,assists_max,boosts_max,damageDealt_max,headshotKills_max,heals_max,killStreaks_max,kills_max,longestKill_max,matchType_max,revives_max,rideDistance_max,roadKills_max,swimDistance_max,teamKills_max,vehicleDestroys_max,walkDistance_max,weaponsAcquired_max,DBNOs_min,Id_min,assists_min,boosts_min,damageDealt_min,headshotKills_min,heals_min,killStreaks_min,kills_min,longestKill_min,matchType_min,revives_min,rideDistance_min,roadKills_min,swimDistance_min,teamKills_min,vehicleDestroys_min,walkDistance_min,weaponsAcquired_min,assists_mean,boosts_mean,damageDealt_mean,DBNOs_mean,headshotKills_mean,heals_mean,kills_mean,killStreaks_mean,longestKill_mean,revives_mean,rideDistance_mean,roadKills_mean,swimDistance_mean,teamKills_mean,vehicleDestroys_mean,walkDistance_mean,weaponsAcquired_mean,assists_match_mean,boosts_match_mean,damageDealt_match_mean,DBNOs_match_mean,headshotKills_match_mean,heals_match_mean,kills_match_mean,killStreaks_match_mean,longestKill_match_mean,revives_match_mean,rideDistance_match_mean,roadKills_match_mean,swimDistance_match_mean,teamKills_match_mean,vehicleDestroys_match_mean,walkDistance_match_mean,weaponsAcquired_match_mean,match_size
0,9329eb41e215eb,676b23c24e70d6,45b576ab7daa7f,0,0,51.46,0,0,0,0,0,0.000000,squad-fpp,0,0.0,0,0.0,0,0,588.000000,1,1,d6267a32c5709c,1,0,74.20,0,0,0,0,0.000000,squad-fpp,0,2694.0,0,0.0,0,0,913.000000,7,0,2f134f2c7be198,0,0,0.00,0,0,0,0,0.000000,squad-fpp,0,0.0,0,0.0,0,0,386.299988,0,0.250000,0.000000,31.415000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,673.500000,0.0,0.0,0.000000,0.0,609.200012,2.500000,0.282609,1.076087,126.950217,0.902174,0.271739,2.195652,0.923913,0.554348,28.244534,0.260870,2245.695557,0.010870,15.673587,0.032609,0.043478,1517.835815,3.630435,92
1,639bd0dcd7bda8,430933124148dd,42a9a0b906c928,0,4,179.10,0,0,2,2,1,361.899994,duo-fpp,2,4669.0,0,0.0,0,0,2017.000000,6,4,ef362b46754f2a,1,6,597.50,1,5,2,5,361.899994,duo-fpp,2,4674.0,0,0.0,0,0,2477.000000,7,0,639bd0dcd7bda8,0,4,179.10,0,2,1,2,192.600006,duo-fpp,0,4669.0,0,0.0,0,0,2017.000000,6,0.500000,5.000000,388.300000,2.000000,0.500000,3.500000,3.500000,1.500000,277.250000,1.000000,4671.500000,0.0,0.0,0.000000,0.0,2247.000000,6.500000,0.187500,1.260417,130.895469,0.604167,0.187500,1.645833,0.916667,0.520833,19.491333,0.125000,1499.132324,0.000000,1.718021,0.041667,0.010417,1147.762573,4.145833,96
2,63d5c8ef8dfe91,0b45f5db20ba99,87e7e4477a048e,1,0,23.40,0,0,4,0,0,0.000000,squad-fpp,0,0.0,0,0.0,0,0,787.799988,4,5,bfe3f0a1c67d92,2,6,1058.00,4,9,2,8,236.800003,squad-fpp,0,2118.0,0,0.0,0,0,2351.000000,8,0,63d5c8ef8dfe91,0,0,23.40,0,0,0,0,0.000000,squad-fpp,0,0.0,0,0.0,0,0,186.300003,1,0.750000,2.250000,372.527500,2.000000,1.000000,3.250000,3.000000,0.750000,63.605000,0.000000,643.125000,0.0,0.0,0.000000,0.0,1099.275024,3.750000,0.297872,1.117021,131.772926,0.797872,0.234043,1.478723,0.893617,0.521277,28.899734,0.202128,1205.909180,0.010638,6.555638,0.021277,0.021277,1093.652588,3.936170,94
3,cf5b81422591d1,b7497dbdc77f4a,1b9a94f1af67f1,0,0,65.52,0,0,0,0,0,0.000000,duo-fpp,0,0.0,0,0.0,0,0,1812.000000,3,1,cf5b81422591d1,0,0,100.00,0,0,0,0,0.000000,duo-fpp,0,0.0,0,0.0,0,0,2116.000000,5,0,271c9c5dd790f4,0,0,65.52,0,0,0,0,0.000000,duo-fpp,0,0.0,0,0.0,0,0,1812.000000,3,0.000000,0.000000,82.760000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,1964.000000,4.000000,0.202247,1.146067,129.342247,0.595506,0.191011,1.359551,0.876404,0.494382,25.816854,0.146067,746.938232,0.000000,0.000000,0.056180,0.022472,1482.995605,3.820225,89
4,ee6a295187ba21,6604ce20a1d230,40754a93016066,0,4,330.20,1,2,1,3,1,60.060001,squad-fpp,1,0.0,0,0.0,0,0,2963.000000,4,2,ee6a295187ba21,1,4,330.20,2,2,1,3,110.900002,squad-fpp,1,0.0,0,0.0,0,0,3027.000000,9,1,6d2de7

In [9]:
df_train_cvt = df_train.copy()

In [10]:
# 회귀분석을 하기 위해 문자열을 범주값으로 변환
matchTyp = {'squad-fpp': 3, 'duo': 2, 'solo-fpp': 1, 'squad': 3, 'duo-fpp': 2, 'solo': 1,
       'normal-squad-fpp': 4, 'crashfpp': 4, 'flaretpp': 4, 'normal-solo-fpp': 4,
       'flarefpp': 4, 'normal-duo-fpp': 4, 'normal-duo': 4, 'normal-squad': 4,
       'crashtpp': 4, 'normal-solo': 4 }

df_train_cvt['matchType'] = df_train_cvt['matchType'].replace(matchTyp)

#필요 없는 칼럼 제거
del_list = ['assists', 'boosts','damageDealt', 'DBNOs', 'teamKills','roadKills','vehicleDestroys',
       'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
       'revives', 'rideDistance', 'swimDistance', 'walkDistance',
       'weaponsAcquired','matchType_max','matchType_min','Id_max','Id_min',]

for col in del_list :
    del df_train_cvt[col]
    
# # null값 확인 및 
# # inplace=False로 하면 기존 혹은 새로운 변수에 할당해야 하고, inplace = True 하면 해당변수에 적용됨
# # na가 포함된 행을 제거하는 것은 데이터 소실이 크기 때문에 inplace=False가 default
# df_train_cvt.dropna(inplace = True)
# print(df_train_cvt.isnull().any().any())

In [11]:
data = df_train_cvt.copy()

In [12]:
data.columns

Index(['Id', 'groupId', 'matchId', 'matchType', 'DBNOs_max', 'assists_max',
       'boosts_max', 'damageDealt_max', 'headshotKills_max', 'heals_max',
       'killStreaks_max', 'kills_max', 'longestKill_max', 'revives_max',
       'rideDistance_max', 'roadKills_max', 'swimDistance_max',
       'teamKills_max', 'vehicleDestroys_max', 'walkDistance_max',
       'weaponsAcquired_max', 'DBNOs_min', 'assists_min', 'boosts_min',
       'damageDealt_min', 'headshotKills_min', 'heals_min', 'killStreaks_min',
       'kills_min', 'longestKill_min', 'revives_min', 'rideDistance_min',
       'roadKills_min', 'swimDistance_min', 'teamKills_min',
       'vehicleDestroys_min', 'walkDistance_min', 'weaponsAcquired_min',
       'assists_mean', 'boosts_mean', 'damageDealt_mean', 'DBNOs_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'revives_mean', 'rideDistance_mean',
       'roadKills_mean', 'swimDistance_mean', 'teamKills_mean',
       've

In [13]:
data['total_distance_max'] = data['rideDistance_max']+data['swimDistance_max']+data['walkDistance_max']

In [14]:
data['total_distance_min'] = data['rideDistance_min']+data['swimDistance_min']+data['walkDistance_min']

In [15]:
data['total_distance_mean'] = data['rideDistance_mean']+data['swimDistance_mean']+data['walkDistance_mean']

In [16]:
data['total_distance_match_mean'] = data['rideDistance_match_mean']+data['swimDistance_match_mean']+data['walkDistance_match_mean']

In [17]:
del data['rideDistance_min']
del data['swimDistance_min']
del data['walkDistance_min']
del data['rideDistance_max']
del data['swimDistance_max']
del data['walkDistance_max']
del data['rideDistance_mean']
del data['swimDistance_mean']
del data['walkDistance_mean']
del data['rideDistance_match_mean']
del data['swimDistance_match_mean']
del data['walkDistance_match_mean']

In [18]:
data.head()

,Id,groupId,matchId,matchType,DBNOs_max,assists_max,boosts_max,damageDealt_max,headshotKills_max,heals_max,killStreaks_max,kills_max,longestKill_max,revives_max,roadKills_max,teamKills_max,vehicleDestroys_max,weaponsAcquired_max,DBNOs_min,assists_min,boosts_min,damageDealt_min,headshotKills_min,heals_min,killStreaks_min,kills_min,longestKill_min,revives_min,roadKills_min,teamKills_min,vehicleDestroys_min,weaponsAcquired_min,assists_mean,boosts_mean,damageDealt_mean,DBNOs_mean,headshotKills_mean,heals_mean,kills_mean,killStreaks_mean,longestKill_mean,revives_mean,roadKills_mean,teamKills_mean,vehicleDestroys_mean,weaponsAcquired_mean,assists_match_mean,boosts_match_mean,damageDealt_match_mean,DBNOs_match_mean,headshotKills_match_mean,heals_match_mean,kills_match_mean,killStreaks_match_mean,longestKill_match_mean,revives_match_mean,roadKills_match_mean,teamKills_match_mean,vehicleDestroys_match_mean,weaponsAcquired_match_mean,match_size,total_distance_max,total_distance_min,total_distance_mean,total_distance_match_mean
0,9329eb41e215eb,676b23c24e70d6,45b576ab7daa7f,3,1,1,0,74.2,0,0,0,0,0.000000,0,0,0,0,7,0,0,0,0.00,0,0,0,0,0.000000,0,0,0,0,0,0.250000,0.000000,31.415000,0.250000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,2.50,0.282609,1.076087,126.950217,0.902174,0.271739,2.195652,0.923913,0.554348,28.244534,0.260870,0.010870,0.032609,0.043478,3.630435,92,3607.0,386.299988,1282.699951,3779.205078
1,639bd0dcd7bda8,430933124148dd,42a9a0b906c928,2,4,1,6,597.5,1,5,2,5,361.899994,2,0,0,0,7,0,0,4,179.10,0,2,1,2,192.600006,0,0,0,0,6,0.500000,5.000000,388.300000,2.000000,0.5,3.500000,3.500000,1.50,277.250000,1.000000,0.0,0.0,0.0,6.50,0.187500,1.260417,130.895469,0.604167,0.187500,1.645833,0.916667,0.520833,19.491333,0.125000,0.000000,0.041667,0.010417,4.145833,96,7151.0,6686.000000,6918.500000,2648.612793
2,63d5c8ef8dfe91,0b45f5db20ba99,87e7e4477a048e,3,5,2,6,1058.0,4,9,2,8,236.800003,0,0,0,0,8,0,0,0,23.40,0,0,0,0,0.000000,0,0,0,0,1,0.750000,2.250000,372.527500,2.000000,1.0,3.250000,3.000000,0.75,63.605000,0.000000,0.0,0.0,0.0,3.75,0.297872,1.117021,131.772926,0.797872,0.234043,1.478723,0.893617,0.521277,28.899734,0.202128,0.010638,0.021277,0.021277,3.936170,94,4469.0,186.300003,1742.400024,2306.117432
3,cf5b81422591d1,b7497dbdc77f4a,1b9a94f1af67f1,2,1,0,0,100.0,0,0,0,0,0.000000,0,0,0,0,5,0,0,0,65.52,0,0,0,0,0.000000,0,0,0,0,3,0.000000,0.000000,82.760000,0.500000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,4.00,0.202247,1.146067,129.342247,0.595506,0.191011,1.359551,0.876404,0.494382,25.816854,0.146067,0.000000,0.056180,0.022472,3.820225,89,2116.0,1812.000000,1964.000000,2229.933838
4,ee6a295187ba21,6604ce20a1d230,40754a93016066,3,2,1,4,330.2,2,2,1,3,110.900002,1,0,0,0,9,1,0,1,24.54,0,1,1,1,2.878000,0,0,0,0,4,0.333333,2.333333,220.613333,1.333333,1.0,1.666667,2.333333,1.00,57.945999,0.333333,0.0,0.0,0.0,6.00,0.273684,0.936842,121.609368,0.926316,0.178947,1.231579,0.905263,0.547368,22.598915,0.252632,0.000000,0.042105,0.000000,3.442105,95,3027.0,2705.000000,2898.333252,1145.260132


In [19]:
data = round(data,4)

In [20]:
data.to_csv('./data/final3/final3_Data_test(version3).csv',sep=',', encoding='utf-8', index=False)